This program is free software; you can redistribute it and/or modify it under the terms of the GNU General Public License, version 2, as published by the Free Software Foundation.

This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

Author: Alexander Young

### This script converts age grids to sea floor bathymetry using the lithospheric cooling models of Richards et al., (2018). It also calculates the volume above subduction zones from the total topography output from geodynamic models.

##### Input:
Sea floor age grid e.g. agegrid_final_mask_0.grd

Total topography grid e.g. C1TT_MantleFrameGrid0.grd

reconstructed subduction zone segments e.g. topology_subduction_boundaries_0.00Ma.xy

inverted etopo bathymetry grid e.g. topo_ice_6m_sz_100km_negative_only_inverted_sign_orig.nc

##### Output:
paleobathymetry grid (excluding subduction zones) e.g. Pk_bathy_sz_300km_0Ma.nc

trench volume through time e.g. C1TT_sz_300_km_normalised_volume_trench_age.dat

##### Citations:
Richards, F.D., M.J. Hoggard, L.R. Cowton & N.J. White (2018). Reassessing the thermal structure of oceanic lithosphere with revised global inventories of basement depths and heat flow measurements, Journal of Geophysical Research: Solid Earth, Vol. 123, 9136-9161.

In [30]:
import sys
import os
import os.path
import subprocess
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd

# allow plots to appear within the notebook
% matplotlib inline

In [40]:
# -- set input variables
model='C1'
topoType='TT'

# -- set stencil width
stencilWidth=300

# -- path to model topography grids
workDir="/Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/GPlatesFiles/%s/TotalTopography" %(model)

# -- path to output directories -- make outDir directories if they do not exist
bathyDir="/Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/BathymetryModeling"
if not os.path.exists(bathyDir):
    os.makedirs(bathyDir)

# path to bathymetry grid files
outDir=bathyDir+ "/%s_Volumes" %(model)
if not os.path.exists(outDir):
    os.makedirs(outDir)

# path to subduction zone topography girds
szBathyGridDir=outDir+"/SZ_topographyGrids_%s" %(model)
if not os.path.exists(szBathyGridDir):
    os.makedirs(szBathyGridDir)
    
# path to subduction zone stencil directory
stencilDir=outDir+"/%s_SZ_stencils" %(model)
if not os.path.exists(stencilDir):
    os.makedirs(stencilDir)

# -- path to exported model topologies (GMT format *.xy)
topologyDir="/Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/GPlatesFiles/TopologiesExport"

# -- set trench volume through time output file name    
volumeFile=outDir+"/%s%s_sz_%d_km_volume_trench_age_new.dat" %(model, topoType, stencilWidth)
VF=open(volumeFile, 'w')

# -- path to model time file
timeFile=workDir+'/%s%s.timese' %(model, topoType)

# -- read time file
f=open(timeFile)

Age_List=[]
Step_List=[]
for line in f:
    parts = line.split() # split line into parts
    if len(parts) > 1:   # if at least 2 parts/columns
        Step_List.append(parts[0])
        Age_List.append(parts[1])


In [34]:
# calculate the volume above trenches from geodynamic model total topography.

# -- create time file loop
for i in range(len(Age_List)):
    age=int(Age_List[i])
    step=int(Step_List[i])
    
    # Path for input: 
    # subduction topologies, topography from dynamic model, and subduction zone stencil grid (will be made if needed)
    szFile=topologyDir+"/topology_subduction_boundaries_%d.00Ma.xy" %(age)
    topoGrid=workDir+"/%s%s.topo_corr.%d.grd" %(model, topoType, step)
    stencilGrid=stencilDir+"/%s_stencil_mask_%dMa_%dkm.nc" %(model, age, stencilWidth)
    
    # Path for output
    # masked topography grids from dynamic model
    topoGridStencil=outDir+"/%s%s_sz_%dMa_%dkm.nc" %(model, topoType, age, stencilWidth)
    topoGridStencilNegative=szBathyGridDir+"/%s%s_sz_%dMa_%dkm_negative_only.nc" %(model, topoType, age, stencilWidth)
    topoGridStencilNegativeInvertedSign=outDir+"/%s%s_sz_%dMa_%dkm_negative_only_inverted_sign.nc" %(model, topoType, age, stencilWidth)
    
    # resample topogrid
    cmd="grdsample %s -I1d -Gtopo_resample.nc" %(topoGrid)
    print cmd
    os.system(cmd)

    # -- make SZ stencil grid if one does not already exist
    cmd="grdmask %s -Rg -I1d -m -NNaN/1/1 -S%dfk -G%s" %(szFile, stencilWidth, stencilGrid)
    print cmd
    os.system(cmd)
    
    # -- use SZ stencil to extract topography values from dynamic model
    cmd="grdmath topo_resample.nc %s MUL -V = %s" %(stencilGrid, topoGridStencil)
    print cmd
    os.system(cmd)
    
    # -- tidy up topography stencil grid
    cmd="grdclip %s -G%s -Sa-2216/NaN -V" %(topoGridStencil, topoGridStencilNegative)
    print cmd
    os.system(cmd)
    
    # -- invert topography stencil grid values
    cmd="grdmath %s -1 MUL = %s" %(topoGridStencilNegative, topoGridStencilNegativeInvertedSign)
    print cmd
    os.system(cmd)

    # -- calculate volume of topography grid
    cmd="grdvolume -Sk %s -Rg -C0 | awk '{print $3}'" %(topoGridStencilNegativeInvertedSign)
    print cmd
    # -- convert from volume in km to m
    volumeTrench=(float(os.popen(cmd).read()))*1e6
    
    # -- report volume above sz across stencil width length according to topography grid 
    print age, volumeTrench
    
    # -- tidy up directory
    cmd="rm -v %s %s" %(topoGridStencil, topoGridStencilNegativeInvertedSign)
    print cmd
    os.system(cmd)
    
    # -- write results to output file
    VF.write("%d %E\n" % (age, volumeTrench))

VF.close()    

grdsample /Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/GPlatesFiles/C1/TotalTopography/C1TT.topo_corr.16015.grd -I1d -Gtopo_resample.nc
grdmask /Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/GPlatesFiles/TopologiesExport/topology_subduction_boundaries_560.00Ma.xy -Rg -I1d -m -NNaN/1/1 -S300fk -G./BathymetryModeling/C1_Volumes/C1_SZ_stencils/C1_stencil_mask_560Ma_300km.nc
grdmath topo_resample.nc ./BathymetryModeling/C1_Volumes/C1_SZ_stencils/C1_stencil_mask_560Ma_300km.nc MUL -V = ./BathymetryModeling/C1_Volumes/C1TT_sz_560Ma_300km.nc
grdclip ./BathymetryModeling/C1_Volumes/C1TT_sz_560Ma_300km.nc -G./BathymetryModeling/C1_Volumes/SZ_topographyGrids_C1/C1TT_sz_560Ma_300km_negative_only.nc -Sa-2216/NaN -V
grdmath ./BathymetryModeling/C1_Volumes/SZ_topographyGrids_C1/C1TT_sz_560Ma_300km_negative_only.nc -1 MUL = ./BathymetryModeling/C1_Volumes/C1TT_sz_560Ma_300km_negative_only_inverted_sign.nc
grdvolume -Sk ./BathymetryModeling/C1_Volumes/C1TT_sz_560Ma_300km_negative_only_inverted_sign.nc -Rg

grdmath ./BathymetryModeling/C1_Volumes/SZ_topographyGrids_C1/C1TT_sz_420Ma_300km_negative_only.nc -1 MUL = ./BathymetryModeling/C1_Volumes/C1TT_sz_420Ma_300km_negative_only_inverted_sign.nc
grdvolume -Sk ./BathymetryModeling/C1_Volumes/C1TT_sz_420Ma_300km_negative_only_inverted_sign.nc -Rg -C0 | awk '{print $3}'
420 7.39422390298e+16
rm -v ./BathymetryModeling/C1_Volumes/C1TT_sz_420Ma_300km.nc ./BathymetryModeling/C1_Volumes/C1TT_sz_420Ma_300km_negative_only_inverted_sign.nc
grdsample /Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/GPlatesFiles/C1/TotalTopography/C1TT.topo_corr.23577.grd -I1d -Gtopo_resample.nc
grdmask /Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/GPlatesFiles/TopologiesExport/topology_subduction_boundaries_400.00Ma.xy -Rg -I1d -m -NNaN/1/1 -S300fk -G./BathymetryModeling/C1_Volumes/C1_SZ_stencils/C1_stencil_mask_400Ma_300km.nc
grdmath topo_resample.nc ./BathymetryModeling/C1_Volumes/C1_SZ_stencils/C1_stencil_mask_400Ma_300km.nc MUL -V = ./BathymetryModeling/C1_Volumes/C1TT_

grdmask /Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/GPlatesFiles/TopologiesExport/topology_subduction_boundaries_280.00Ma.xy -Rg -I1d -m -NNaN/1/1 -S300fk -G./BathymetryModeling/C1_Volumes/C1_SZ_stencils/C1_stencil_mask_280Ma_300km.nc
grdmath topo_resample.nc ./BathymetryModeling/C1_Volumes/C1_SZ_stencils/C1_stencil_mask_280Ma_300km.nc MUL -V = ./BathymetryModeling/C1_Volumes/C1TT_sz_280Ma_300km.nc
grdclip ./BathymetryModeling/C1_Volumes/C1TT_sz_280Ma_300km.nc -G./BathymetryModeling/C1_Volumes/SZ_topographyGrids_C1/C1TT_sz_280Ma_300km_negative_only.nc -Sa-2216/NaN -V
grdmath ./BathymetryModeling/C1_Volumes/SZ_topographyGrids_C1/C1TT_sz_280Ma_300km_negative_only.nc -1 MUL = ./BathymetryModeling/C1_Volumes/C1TT_sz_280Ma_300km_negative_only_inverted_sign.nc
grdvolume -Sk ./BathymetryModeling/C1_Volumes/C1TT_sz_280Ma_300km_negative_only_inverted_sign.nc -Rg -C0 | awk '{print $3}'
280 5.98609710994e+16
rm -v ./BathymetryModeling/C1_Volumes/C1TT_sz_280Ma_300km.nc ./BathymetryModeling/C1_Volu

grdmath ./BathymetryModeling/C1_Volumes/SZ_topographyGrids_C1/C1TT_sz_140Ma_300km_negative_only.nc -1 MUL = ./BathymetryModeling/C1_Volumes/C1TT_sz_140Ma_300km_negative_only_inverted_sign.nc
grdvolume -Sk ./BathymetryModeling/C1_Volumes/C1TT_sz_140Ma_300km_negative_only_inverted_sign.nc -Rg -C0 | awk '{print $3}'
140 7.91315189457e+16
rm -v ./BathymetryModeling/C1_Volumes/C1TT_sz_140Ma_300km.nc ./BathymetryModeling/C1_Volumes/C1TT_sz_140Ma_300km_negative_only_inverted_sign.nc
grdsample /Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/GPlatesFiles/C1/TotalTopography/C1TT.topo_corr.35007.grd -I1d -Gtopo_resample.nc
grdmask /Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/GPlatesFiles/TopologiesExport/topology_subduction_boundaries_120.00Ma.xy -Rg -I1d -m -NNaN/1/1 -S300fk -G./BathymetryModeling/C1_Volumes/C1_SZ_stencils/C1_stencil_mask_120Ma_300km.nc
grdmath topo_resample.nc ./BathymetryModeling/C1_Volumes/C1_SZ_stencils/C1_stencil_mask_120Ma_300km.nc MUL -V = ./BathymetryModeling/C1_Volumes/C1TT_

0 8.69974432072e+16
rm -v ./BathymetryModeling/C1_Volumes/C1TT_sz_0Ma_300km.nc ./BathymetryModeling/C1_Volumes/C1TT_sz_0Ma_300km_negative_only_inverted_sign.nc


In [35]:
# Then, use present day trench volume from etopo grid to normalise model trench volume
# Path to present day trench topography grid
etopoFile = "/Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/PythonNotebooks/DependentInputs/topo_ice_6m_sz_%dkm_negative_only_inverted_sign_orig.nc" %(stencilWidth)

# -- calculate volume of topography grid
cmd="grdvolume -Sk %s -Rg | awk '{print $3}'" %(etopoFile)
print cmd
os.system(cmd)

observedVolumeTrench=(float(os.popen(cmd).read()))*1e6

print observedVolumeTrench

# -- set output file name
normalisedVolumeFile=outDir+"/%s%s_sz_%d_km_normalised_volume_trench_age.dat" %(model, topoType, stencilWidth)

# -- open model topography trench volume file from previous step
VF=open(volumeFile, 'r')

# -- split model topography trench volume file into an age list and volume list
Age_List=[]
Volume_List=[]
for line in VF:
    parts = line.split() # split line into parts
    if len(parts) > 1:   # if at least 2 parts/columns
        Age_List.append(int(parts[0]))
        Volume_List.append(float(parts[1]))

VF.close()

#print Age_List[-1], Volume_List[-1]

# -- read through age list and normalise model topography trench volume with present day trench volume  
# -- from topography grid appending normalised volumes to output file
NVF=open(normalisedVolumeFile, 'w')
for i in range(len(Age_List)):
    normalisedVolumeTrench=observedVolumeTrench*Volume_List[i]/Volume_List[-1]
    #print Age_List[i], normalisedVolumeTrench
    NVF.write("%d %.2E\n" % (Age_List[i], normalisedVolumeTrench))

NVF.close() 

grdvolume -Sk /Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/PythonNotebooks/DependentInputs/topo_ice_6m_sz_300km_negative_only_inverted_sign_orig.nc -Rg | awk '{print $3}'
9.39201136102e+16


In [42]:
# Here, we convert the age girds to seafloor bathymetry using either the half-space cooling model, 
# complex plate model or simple plate model from Richards et al., 2018.

# Choose which age depth model to implement
# Toggle age_depth_model to call model for simple plate (zero), half space cooling (1) or
# complex plate (2)
# 0 Pk, 1 HSCk, 2 KRCC

age_depth_model = 0.

# Path to age grid input files
ageGridDir="/Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/AgeGrids/Mask"

# Set model name and create output directories, directories will be made if they do not exist
if age_depth_model == 1.:
    print ("implement Richards et al., 2018 half-space cooling model with constant thermal parameters ")
    coolingModel = 'HSCk'
    szMaskedAgeGridDir=outDir+"/sz_MaskedAgeGrids_%s" %(coolingModel)
    bathyDir=outDir+"/sz_MaskedBathy_%s" %(coolingModel)
    volumeRidgeFile=outDir+"/TotalBathy_volume_%dkm_%s_%s.dat" %(stencilWidth, model, coolingModel)
    print ("running half space cooling model")

elif age_depth_model == 0. :
    print ("implement Richards et al., 2018 simple analytical plate model with constant thermal parameters")
    coolingModel = 'Pk'
    szMaskedAgeGridDir=outDir+"/sz_MaskedAgeGrids_%s" %(coolingModel)
    bathyDir=outDir+"/sz_MaskedBathy_%s" %(coolingModel)
    volumeRidgeFile=outDir+"/TotalBathy_volume_%dkm_%s_%s.dat" %(stencilWidth, model, coolingModel)
    print ("running plate cooling model")

else:
    print ("implement Richards et al., 2018 complete  plate model ")
    coolingModel = 'KRCC'
    szMaskedAgeGridDir=outDir+"/sz_MaskedAgeGrids_%s" %(coolingModel)
    bathyDir=outDir+"/sz_MaskedBathy_%s" %(coolingModel)
    volumeRidgeFile=outDir+"/TotalBathy_volume_%dkm_%s_%s.dat" %(stencilWidth, model, coolingModel)
    print ("running complete plate cooling model")
    
cwd = os.getcwd()

if not os.path.exists(szMaskedAgeGridDir):
    os.makedirs(szMaskedAgeGridDir)

if not os.path.exists(bathyDir):
    os.makedirs(bathyDir)
    
# From here, the model runs
# Path to time file
# timeFile=workDir+'/%s%s.timese' %(model, topoType)
f=open(timeFile, 'r')

# Create time loop from time file
Age_List=[]
for line in f:
    parts = line.split() # split line into parts
    if len(parts) > 1:   # if at least 2 parts/columns
        Age_List.append(parts[1])

# -- set input files at each timestep        
for i in range(len(Age_List)):
    age=int(Age_List[i])
    print ("age is %s Ma" %(age))

    # subduction topologies and masked sz topography values (from dynamic model)
    szFile=topologyDir+"/topology_subduction_boundaries_%d.00Ma.xy" %(age)
    stencilGrid=szBathyGridDir+"/%s%s_sz_%dMa_%dkm_negative_only.nc" %(model, topoType, age, stencilWidth)
    
    # -- path to age grids
    ageGrid=ageGridDir+"/agegrid_final_masked_%s.nc" %(age)
    
    # -- set output file names
    ageGridStencil=szMaskedAgeGridDir+"/MaskedAgeGrid_Final_sz%dkm_%d.nc" %(stencilWidth, age)
    ageGridStencilxyz=outDir+"/MaskedAgeGrid_Final_sz%dkm_%d.xyz" %(stencilWidth, age)
    bathyPrefix="%s_bathy_sz_%dkm_%dMa" %(coolingModel, stencilWidth, age)
    bathyGrid=bathyDir+"/"+bathyPrefix+".nc"
    bathyGridInvertedSign=bathyPrefix+"_InvertedSign.nc"
    
    # -- invert sz stencil and calculate seafloor bathy using agegrids
    # -- this step doesnt automatically wipe the bathy directory and will not run if the directory 
    # -- contains grids already. clear directory before trouble shooting. 
    if not os.path.isfile(bathyGrid):
        
        # -- invert sz stencil
        cmd="grdmath %s ISNAN = tmp1_%d.nc" %(stencilGrid, age)
        print cmd
        os.system(cmd)
        
        # -- clip inverted sz stencil
        cmd="grdclip -V tmp1_%d.nc -Gtmp2_%d.nc -Sb1/NaN -V" %(age, age)
        print cmd
        os.system(cmd)
        
        # -- resample age grid
        cmd="grdsample tmp2_%d.nc -I1d -Gtmp3_%d.nc" %(age, age)
        print cmd
        os.system(cmd)
        
        # -- resample age grid
        cmd="grdsample %s -I1d -Gtmp4_%d.nc" %(ageGrid, age)
        print cmd
        os.system(cmd)
        
        # -- remove age grid values within inverted sz stencil
        cmd="grdmath -V tmp4_%d.nc tmp3_%d.nc OR = %s" %(age, age, ageGridStencil)
        print cmd
        os.system(cmd)
        
        # -- tidy up temp files
        cmd="rm -v tmp1_%d.nc tmp2_%d.nc tmp3_%d.nc tmp4_%d.nc" %(age, age, age, age)
        print cmd
        os.system(cmd)
        
        # -- convert clipped age grid to xyz file
        cmd="grd2xyz %s > %s" %(ageGridStencil, ageGridStencilxyz)
        print cmd
        os.system(cmd)
        
        # -- Load in ageGridStencilxyz file to model depth
        tmp = pd.read_csv(ageGridStencilxyz, delimiter = "\t", header=None, names=['Long', 'Lat', 'age'])
        tmp = tmp[np.isfinite(tmp['age'])]
        tmp['age'] = np.where(tmp['age'] >= 120.0, 120., tmp['age'])
        tmp['age_s'] = tmp.age * 3.1536e13
        
        if age_depth_model == 1.:
            # -- implement Richards et al., 2018 half-space cooling model with constant thermal parameters 
            # -- set input variables
            rhom = 3330. # mantle density, kg/m^3
            rhow = 1030. # ocean water density, kg/m^3
            ap = 3.28e-5 # thermal expansion coefficient, K
            deltaT = 1484. #  temperature difference between the base and top of the oceanic lithosphere, K
            zr = 1656. # depth of ridge, m
            pi = math.pi # phi, no unit
            k = 0.8044 / 1e6 # thermal diffusivity, m^2/s

            def bathy(row):
                return zr + ((2. * rhom * ap * deltaT) / (rhom - rhow)) * (math.sqrt(((k * row['age_s'])/pi)))
            
            tmp['bathy'] = tmp.apply(lambda row: bathy(row), axis=1)
            tmp = tmp.drop(['age', 'age_s'], axis=1)
            tmp['bathy'] *= -1.    
            tmp.to_csv(bathyDir+"/bathy_%dMa_%s.xyz" %(age, age_depth_model), sep=' ', header=False, index=False)
            bathyxyz = bathyDir+"/bathy_%dMa_%s.xyz" %(age, age_depth_model)    

        elif age_depth_model == 0. :
            # -- implement Richards et al., 2018 simple analytical plate model with constant thermal parameters
            # -- set input variables
            index = np.arange(0, 101, 1)
            rhom = 3330. # mantle density, kg/m^3
            rhow = 1030. # ocean water density, kg/m^3
            ap = 3.28e-5 # thermal expansion coefficient, K
            deltaT = 1495. #  temperature difference between the base and top of the oceanic lithosphere, K
            zp = 105000. # thickness of plate, m
            zr = 2216. # depth of ridge, m
            pi = math.pi # phi, no unit
            k = 0.8044 / 1e6 # thermal diffusivity, m^2/s
            print (zr)
            
            def bathy(row):
                x = 0
                for a in index:
                    x += (1. / ((1. + 2. * a)**2.))* (math.exp(-(k*((1.+2.*a)**2.)*(pi**2.)*row['age_s'])/(zp**2.)))
                return (zr + (1. - (8. / pi**2. * x)) * (((rhom * ap * deltaT * zp) / (2. * (rhom -rhow)))))
    
            tmp['bathy'] = tmp.apply(lambda row: bathy(row), axis=1)
            tmp = tmp.drop(['age', 'age_s'], axis=1)
            tmp['bathy'] *= -1.    
            tmp.to_csv(bathyDir+"/bathy_%dMa_%s.xyz" %(age, age_depth_model), sep=' ', header=False, index=False)
            bathyxyz = bathyDir+"/bathy_%dMa_%s.xyz" %(age, age_depth_model)
            
        else:
            # implement Richards et al., 2018 complex plate model
            # import RHCW18 age-depth file
            age_depth_table = pd.read_csv('/Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/PythonNotebooks/DependentInputs/depth-1333-130-2500.dat', sep='  ', names=['age', 'depth'])
            age_depth_table.depth *= -1.
            
            # sort values for merge_asof and use lookup table to convert age to depth 
            tmp1 = tmp.sort_values('age')
            df_x = pd.merge_asof(tmp1, age_depth_table, on = 'age')
            df_x = df_x.drop('age', 1)
            df_x = df_x.drop('age_s', 1)
            df_x.to_csv(bathyDir+"/bathy_%dMa_%s.xyz" %(age, coolingModel), sep=' ', header=False, index=False)
            bathyxyz = bathyDir+"/bathy_%dMa_%s.xyz" %(age, coolingModel)    

        # -- convert seafloor bathy xyz to grd
        cmd="xyz2grd -V %s -Rg -I1d -G%s" %(bathyxyz, bathyGrid)
        print cmd
        os.system(cmd)
        
        # -- tidy up temp files
        cmd="rm -v %s %s %s.xyz" %(ageGridStencil, ageGridStencilxyz, bathyPrefix)
        print cmd
        os.system(cmd)


implement Richards et al., 2018 simple analytical plate model with constant thermal parameters
running plate cooling model
age is 560 Ma
grdmath /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/BathymetryModeling/C1_Volumes/SZ_topographyGrids_C1/C1TT_sz_560Ma_300km_negative_only.nc ISNAN = tmp1_560.nc
grdclip -V tmp1_560.nc -Gtmp2_560.nc -Sb1/NaN -V
grdsample tmp2_560.nc -I1d -Gtmp3_560.nc
grdsample /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/AgeGrids/Mask/agegrid_final_masked_560.nc -I1d -Gtmp4_560.nc
grdmath -V tmp4_560.nc tmp3_560.nc OR = /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/BathymetryModeling/C1_Volumes/sz_MaskedAgeGrids_Pk/MaskedAgeGrid_Final_sz300km_560.nc
rm -v tmp1_560.nc tmp2_560.nc tmp3_560.nc tmp4_560.nc
grd2xyz /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/BathymetryModeling/C1_Volumes/sz_MaskedAgeGrids_Pk/MaskedAgeGrid_Final_sz300km_560.nc > /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/BathymetryModeling/C1_Volumes/Mask

2216.0
xyz2grd -V /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/BathymetryModeling/C1_Volumes/sz_MaskedBathy_Pk/bathy_460Ma_0.0.xyz -Rg -I1d -G/Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/BathymetryModeling/C1_Volumes/sz_MaskedBathy_Pk/Pk_bathy_sz_300km_460Ma.nc
rm -v /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/BathymetryModeling/C1_Volumes/sz_MaskedAgeGrids_Pk/MaskedAgeGrid_Final_sz300km_460.nc /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/BathymetryModeling/C1_Volumes/MaskedAgeGrid_Final_sz300km_460.xyz Pk_bathy_sz_300km_460Ma.xyz
age is 440 Ma
grdmath /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/BathymetryModeling/C1_Volumes/SZ_topographyGrids_C1/C1TT_sz_440Ma_300km_negative_only.nc ISNAN = tmp1_440.nc
grdclip -V tmp1_440.nc -Gtmp2_440.nc -Sb1/NaN -V
grdsample tmp2_440.nc -I1d -Gtmp3_440.nc
grdsample /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/AgeGrids/Mask/agegrid_final_masked_440.nc -I1d -Gtmp4_440.nc
grdmath -V tmp4_440.

grdsample /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/AgeGrids/Mask/agegrid_final_masked_340.nc -I1d -Gtmp4_340.nc
grdmath -V tmp4_340.nc tmp3_340.nc OR = /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/BathymetryModeling/C1_Volumes/sz_MaskedAgeGrids_Pk/MaskedAgeGrid_Final_sz300km_340.nc
rm -v tmp1_340.nc tmp2_340.nc tmp3_340.nc tmp4_340.nc
grd2xyz /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/BathymetryModeling/C1_Volumes/sz_MaskedAgeGrids_Pk/MaskedAgeGrid_Final_sz300km_340.nc > /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/BathymetryModeling/C1_Volumes/MaskedAgeGrid_Final_sz300km_340.xyz
2216.0
xyz2grd -V /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/BathymetryModeling/C1_Volumes/sz_MaskedBathy_Pk/bathy_340Ma_0.0.xyz -Rg -I1d -G/Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/BathymetryModeling/C1_Volumes/sz_MaskedBathy_Pk/Pk_bathy_sz_300km_340Ma.nc
rm -v /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/BathymetryModeling/C1_Vo

grdsample /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/AgeGrids/Mask/agegrid_final_masked_220.nc -I1d -Gtmp4_220.nc
grdmath -V tmp4_220.nc tmp3_220.nc OR = /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/BathymetryModeling/C1_Volumes/sz_MaskedAgeGrids_Pk/MaskedAgeGrid_Final_sz300km_220.nc
rm -v tmp1_220.nc tmp2_220.nc tmp3_220.nc tmp4_220.nc
grd2xyz /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/BathymetryModeling/C1_Volumes/sz_MaskedAgeGrids_Pk/MaskedAgeGrid_Final_sz300km_220.nc > /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/BathymetryModeling/C1_Volumes/MaskedAgeGrid_Final_sz300km_220.xyz
2216.0
xyz2grd -V /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/BathymetryModeling/C1_Volumes/sz_MaskedBathy_Pk/bathy_220Ma_0.0.xyz -Rg -I1d -G/Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/BathymetryModeling/C1_Volumes/sz_MaskedBathy_Pk/Pk_bathy_sz_300km_220Ma.nc
rm -v /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/BathymetryModeling/C1_Vo

grdsample /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/AgeGrids/Mask/agegrid_final_masked_100.nc -I1d -Gtmp4_100.nc
grdmath -V tmp4_100.nc tmp3_100.nc OR = /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/BathymetryModeling/C1_Volumes/sz_MaskedAgeGrids_Pk/MaskedAgeGrid_Final_sz300km_100.nc
rm -v tmp1_100.nc tmp2_100.nc tmp3_100.nc tmp4_100.nc
grd2xyz /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/BathymetryModeling/C1_Volumes/sz_MaskedAgeGrids_Pk/MaskedAgeGrid_Final_sz300km_100.nc > /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/BathymetryModeling/C1_Volumes/MaskedAgeGrid_Final_sz300km_100.xyz
2216.0
xyz2grd -V /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/BathymetryModeling/C1_Volumes/sz_MaskedBathy_Pk/bathy_100Ma_0.0.xyz -Rg -I1d -G/Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/BathymetryModeling/C1_Volumes/sz_MaskedBathy_Pk/Pk_bathy_sz_300km_100Ma.nc
rm -v /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/BathymetryModeling/C1_Vo